In [ ]:
import numpy as np
import pandas as pd
import scimap as sm
import scanpy as sc
import anndata as ad
import os
import shutil

In [ ]:
img_name = "TS-373_IMC21_UB_001"
adata = ad.read_h5ad(f"/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/phenotyping/phenotyping_outputs/uncorrected/standard/{img_name}standard.h5ad")
adata_old = ad.read_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/old_preprocessed/cells.h5ad")
channel_names = list(adata_old.var_names)
adata.uns['all_markers'] = channel_names

In [ ]:
adata.obs

In [ ]:
adata.obs['phenotype'].value_counts()

In [ ]:
adata2 = ad.read_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_annotated_pp_osteocytes_cleaned_nbh.h5ad")
adata2.uns['all_markers'] = channel_names
adata2 = adata2[adata2.obs['image_ID'] == f'{img_name}.csv']
adata2.obs['Phenotype3'].value_counts()

In [ ]:
subset = img_name
image_path = f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/img/{subset}.tiff'
mask_path = f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/masks/{subset}.tiff'
sm.pl.image_viewer(image_path = image_path, adata = adata, overlay = 'phenotype', point_color='white', imageid = 'imageid', point_size=5, seg_mask = mask_path)

In [ ]:
sm.pl.image_viewer(image_path = image_path, adata = adata2, overlay = 'Phenotype3', point_color='white', imageid = 'imageid', point_size=5, seg_mask = mask_path)

In [ ]:
df = pd.read_csv(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/merged_csv/{img_name}.csv', index_col=0)

In [ ]:
adata2

In [ ]:
df.rename(columns={'centroid-0': 'y', 'centroid-1': 'x'}, inplace=True)
df['image_id'] = img_name

In [ ]:
phenotypes = adata2.obs[['Phenotype3', 'Object']]
phenotypes.reset_index(inplace=True)
phenotypes.drop(columns='index', inplace=True)
phenotypes

In [ ]:
df2 = pd.merge(df, phenotypes, on='Object', how='inner')
df2.rename(columns={'Phenotype3': 'phenotype', 'Object': 'Cell_ID'}, inplace=True)
cell_id_col = df2.pop('Cell_ID')
df2.insert(len(df2.columns) - 1, 'Cell_ID', cell_id_col)
df2

In [ ]:
df2.to_csv(f'/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/myeloma_IMC/quantification/{img_name}.csv', index=True)
shutil.copy(image_path, f'/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/myeloma_IMC/raw_images/multistack_tiffs/{img_name}.tiff')
shutil.copy(mask_path, f'/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/myeloma_IMC/segmentation/{img_name}.tiff')

In [ ]:
# Good quality images and phenotypes checked, now creating the combined csv based on csv names
adata = ad.read_h5ad('/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_annotated_pp_osteocytes_cleaned_nbh_macs.h5ad')


In [ ]:
lst = []
for i, csv in enumerate(os.listdir('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/myeloma_IMC/quantification/')):
    if not (csv.startswith('.') or csv.startswith('myeloma')) and csv.endswith('.csv'):
        #i = pd.read_csv(f'/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/myeloma_IMC/quantification/{csv}')
        df = adata[adata.obs['image_ID'] == csv].to_df()
        df2 = adata[adata.obs['image_ID'] == csv].obs
        i = pd.merge(df, df2, left_index=True, right_index=True)
        i.rename(columns={'Phenotype4': 'phenotype', 'Object': 'CellID', 'X_centroid': 'x', 'Y_centroid': 'y'}, inplace=True)
        i.drop(columns=['Phenotype', 'Phenotype2', 'Phenotype3', 'HLA-DR_counts', 'distance_to_bone'], inplace=True)
        lst.append(i)
comb = pd.concat(lst)
comb

In [ ]:
comb.columns

In [ ]:
comb.to_csv('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/myeloma_IMC/quantification/myeloma_IMC_combined.csv', index=False)

## Harmonize celltype labels

In [ ]:
df = pd.read_csv('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/myeloma_IMC_quantification.csv')
df

In [ ]:
df.columns

In [ ]:
df['cell_type'].value_counts()

In [ ]:
df['cell_type'] = df['cell_type'].replace({'Plasma Cells/MM cells': 'Plasma_cell', 'Neutrophils': 'Neutrophil', 'Unknown':'undefined', 'CD8+Tcells':'CD8+_T_cell',
                                           'CD4+Tcells':'CD4+_T_cell', 'activated Macrophages/Monocytes': 'Macrophage', 'Macrophages/Monocytes': 'Macrophage',
                                           'Endothelial cells': 'Endothelial', 'Adipocytes': 'Adipocyte', 'Dendritic Cells': 'Dendritic_cell', 'Osteocytes': 'Osteocyte',
                                           'Osteoclasts': 'Osteoclast', 'Osteoblasts': 'Osteoblast'})

In [ ]:
df.to_csv('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/myeloma_IMC_quantification.csv', index=False)